In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Download Libraries


In [ ]:
!pip install transformers
!pip install groq  wandb
!pip install json_repair

In [ ]:
!pip install -qU faker
!pip install -qU vllm
!pip install json_repair

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

In [ ]:
from google.colab import userdata
import wandb

wandb.login(key = userdata.get('wandb'))
hf_token = userdata.get('huggingface')
!huggingface-cli login --token {hf_token}

## Import Libraries

In [ ]:
import json
import os
from os.path import join
import requests
import random
from tqdm.auto import  tqdm

# for Creating Schema
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime
import json_repair

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

import time
# from groq import APIStatusError

In [3]:
# Links
data_dir = '/gdrive/MyDrive/FineTuning_fromScratch'
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"

def repaire_json(article_text):
  try:
    return json_repair.loads(article_text)
  except:
    return None

In [4]:
article = """
في أبريل/نيسان 2022، دخل شخصان مكتبة جامعة تارتو في إستونيا، وطلبا الاطلاع على ثمانية كتب لفنانين من القرن التاسع عشر: الشاعر والكاتب المسرحي الروسي ألكسندر بوشكين، والكاتب الروسي الشهير من أصل أوكراني نيكولاي غوغول.

تحدث الرجلان بالروسية، مؤكدين لموظف المكتبة أن الأصغر سناً منهما يُجري بحثاً لإكمال دراسته في الولايات المتحدة.

بعد ثلاثة أشهر، لاحظ موظفو المكتبة استبدال كتابين من الكتب الثمانية بنسخ مزيفة. وكشفت سجلات المكتبة أن الكتابين الأصليين كانا قد خرجا للمرة الأخيرة بحوزة نفس الرجلين في أبريل/نيسان.

سارع أمناء المكتبة إلى فحص الكتب الستة الأخرى التي تعد من الكلاسيكيات الروسية. وبعد فحص دقيق، تبيّن أن جميعها مزيفة بطريقة متقنة، إذ كانت تبدو أصلية لوجود أختام المكتبة وأرقام الجرد عليها.

لم تكن مكتبة تارتو هي الضحية الوحيدة. فبعد أسابيع، اختفت أيضاً عشرة كتب نادرة من مكتبة جامعة تالين في العاصمة الإستونية.

على مدار 18 شهراً، سُرقت طبعات أصلية لكتب كلاسيكية روسية وغيرها من المطبوعات الأثرية باللغة الروسية من عشرات المكتبات الأوروبية في دول البلطيق وفنلندا وسويسرا وفرنسا.

في بعض الحالات، استُبدلت النسخ الأصلية بنسخ مزيفة، وفي حالات أخرى، خرجت الكتب من المكتبة ولم تعد إليها أبداً.

وفي استجابة لتلك الحوادث، أطلقت وكالة الاتحاد الأوروبي للتعاون في مجال إنفاذ القانون (يوروبول) تحقيقاً يسمى بـ"عملية بوشكين"، نسبةً إلى سرقة أعمال الكاتب المسرحي الروسي. وشارك في التحقيق أكثر من 100 ضابط، إذ قاموا بتفتيش عدد من الممتلكات في عدة دول أوروبية. وأسفرت الحملة حتى الآن عن اعتقال تسعة من المشتبه بهم، جميعهم يحملون الجنسية الجورجية.

كان بيكا تسيريكيدزه البالغ 48 عاماً، أول مشتبه به يُقبض عليه. حُوكم وأُدين بارتكاب ثلاث جرائم في بلدين مختلفين في نفس الوقت، لاتفيا وإستونيا، بينها سرقات من مكتبتي تارتو وتالين.

يقضي تسيريكيدزه حالياً عقوبة السجن لمدة ثلاث سنوات وثلاثة أشهر في إستونيا، حيث يُسمح للسجناء بالتحدث إلى الصحفيين.

وفي حديثه لبي بي سي قال تسيريكيدزه، إنه قرر عام 2008 الاتجار في الكتب القديمة، من خلال شرائها وترميمها وإعادة بيعها، لتوفير احتياجات أسرته.

وعن التعليم الذي تلقاه للعمل كمرمم كتب، أوضح أنه "تعلم كل شيء من خلال الممارسة".

وشبّه نفسه بأنه مثل "ساحر" مع الكتب. مضيفاً: "بمجرد أن أُمسك الكتاب أستطيع أن أحدد فوراً قيمته والمبلغ الذي قد يدفعونه في المزاد من أجله".

كانت أول مواجهة لتسيريكيدزه مع القانون في عام 2016 عندما أُدين في جورجيا بتهمة سرقة كتب قديمة من متحف تبليسي التاريخي. وقد اعترف بذنبه في ذلك الوقت، ليصدر ضده حكماً بالسجن مع وقف التنفيذ.

"أكبر عملية سرقة منذ الحرب العالمية الثانية"

ي أكتوبر/تشرين الأول 2023، جلس زوجان شابان في قاعة المطالعة بمكتبة جامعة وارسو: ارتدى الرجل قبعة بيسبول سوداء وكانت المرأة ذات شعر أحمر.

كان يتصفحان كتباً قديمة في المكتبة، وفي لحظة ما، كان الرجل يُقبّل رفيقته على خدها. ظهر كل هذا في تسجيل من كاميرا مراقبة.

تبيّن أن الشاب هو ماتي، ابن تسيريكيدزه، أما المرأة فهي آنا غوغولادزي، زوجة ماتي.

لاحقاً قبضت الشرطة عليهما بتهمة سرقة كتب من المكتبة، تُقدر قيمتها بحوالي 100 ألف دولار، وأُدينا بالجريمة.

تمتلك جامعة وارسو مجموعة ثرية جداً من كتب ما قبل وما بعد الاتحاد السوفيتي. وكانت هذه المطبوعات قد نجت بأعجوبة من انتفاضة وارسو عام 1944، عندما احترق مبنى الجامعة نفسه.

قال البروفيسور هيرونيم غرالا، من جامعة وارسو لبي بي سي: "نحن الجيل الذي يدرك تماماً فكرة أن أحدهم حافظ على هذه الكتب من أجلنا ذات يوم".

خلال أقل من عام، بلغ إجمالي السرقات من مكتبة الجامعة 73 نسخة نادرة، تُقدر قيمتها بنحو 600 ألف دولار. ولم يُقبض على بعض الجناة حتى الآن.

وصف البروفيسور غرالا حجم الجرائم لوسائل الإعلام، بأنها "أكبر سرقة منذ الحرب العالمية الثانية. إنها تشبه تحطيم جوهرة تاج".

لم يلاحظ موظفو المكتبة استبدال الكتب الأصلية بالمزيفة على الفور. قال البروفيسور غرالا: "إن الكتب المزيفة كانت تحمل الرمز نفسه (الكود)، وبنفس حجم النسخ الأصلية، ولم تكن هناك كتب مفقودة من على الرفوف".

وقال البروفيسور غرالا لبي بي سي: "لا شك أن مجموعة اللصوص التي وصلت في البداية (كانت تستهدف كتباً محددة) وأعدت نفسها جيداً. ويبدو أنهم صنعوا نُسخاً عالية الجودة".

وأشار البروفيسور غرالا إلى أن الإصلاحات المكتبية الأخيرة التي خضعت لها هذه المكتبة ربما كانت السبب في تزايد عدد السرقات، خاصة تلك المتعلقة بتخفيف قواعد الفحص للكتب النادرة والقديمة.

وتهدف هذه الإجراءات في الأصل إلى زيادة سهولة وصول القُرّاء للكتب.

"""

## Details Extraction

In [5]:
ArticleCategory =Literal[
       "politics", "sports", "art", "technology", "economy",
      "health", "entertainment", "science",
      "not_specified"
]
EntityType = Literal[
        "person-male", "person-female", "location", "organization", "event", "time",
      "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]

class Entity(BaseModel):
  entity_value: str = Field(..., description = "The actual name or value of the entity.")
  entity_type: EntityType = Field(..., description= "The type of recognized entity.")


class ArticleDetails(BaseModel):
  article_title: str = Field(..., min_length=5, max_length=100, description= "A fully informative and SEO optimized title of the article.")
  article_date: datetime = Field(..., description= "The date and time when the article was published.")
  article_keywords: List[str] = Field(..., min_items =1, description= "Relevant keywords associated with the story.")
  article_summary: List[str] = Field(..., min_items = 1, max_items = 10, description= "Summarized key points about the story (1-10 points).")
  article_category: ArticleCategory = Field(..., description= "The category of the article.")
  article_entity: List[Entity] = Field(..., min_items=1, max_items=10 ,description= "The entity associated with the article.")



# 2 role system and user
details_extraction_messages = [
    {
        "role":"system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role":"user",
        "content": "\n".join([
            "## Article:",
            article.strip(),
            "",

            "## Pydantic Scheme:",
            json.dumps(ArticleDetails.model_json_schema(), ensure_ascii=False),
            "",

            "## Article Details:",
            "```json"
        ])
    }
]


## Translation

In [6]:
class Translated_Article(BaseModel):
  translation_title: str = Field(...,min_length = 10, max_length = 50 ,description= "Suggested translated title of the article.")
  translation_content: str = Field(..., min_items = 5, description= "Translated content of the article")


targted_language = "English"

translation_messages =[
    {
        "role":"system",
          "content": "\n".join([
            "You are a professional translator.",
            "You will be provided by an Arabic text.",
            "You have to translate the text into the `Targeted Language`.",
            "Follow the provided Scheme to generate a JSON",
            "Do not generate any introduction or conclusion."
          ])
    },
    {
      "role":"user",
      "content": "\n".join([
            "## Article:",
            article.strip(),
            "",

            "## Pydantic Scheme:",
            json.dumps(Translated_Article.model_json_schema(), ensure_ascii=False),
            "",

            "## Targeted Language:",
            targted_language,
            "",

            "## Article Details:",
            "```json"
      ])

    }
]

## Evaluation

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype= None)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
model.save_pretrained("/content/sample_data/Qwen_Model")
tokenizer.save_pretrained("/content/sample_data/Tokenizer_Model")

In [ ]:
# For Details Extraction

text = tokenizer.apply_chat_template(details_extraction_messages, tokenize=False, add_generation_prompt=True)

model_inputs = tokenizer([text],  return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=1024,
                               do_sample = False,  top_k=None, temperature=None, top_p=None)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(generated_text)

```json
{
  "article_title": "أكبر عملية سرقة منذ الحرب العالمية الثانية",
  "article_date": "2023-10-01T00:00:00Z",
  "article_keywords": [
    "سرقة",
    "كتب",
    "大学图书馆",
    "اتحاد الأوروبي للتعاون في مجال إنفاذ القانون"
  ],
  "article_summary": [
    "في أبريل/نيسان 2022، دخل شخصان مكتبة جامعة تارتو في إستونيا، وطلبا الاطلاع على ثمانية كتب لفنانين من القرن التاسع عشر.",
    "تحدث الرجلان بالروسية، مؤكدين لموظف المكتبة أن الأصغر سناً منهما يُجري بحثاً لإكمال دراسته في الولايات المتحدة.",
    "بعد ثلاثة أشهر، لاحظ موظفو المكتبة استبدال كتابين من الكتب الثمانية بنسخ مزيفة. وكشفت سجلات المكتبة أن الكتابين الأصليين كانا قد خرجا للمرة الأخيرة بحوزة نفس الرجلين في أبريل/نيسان.",
    "سارع أمناء المكتبة إلى فحص الكتب الستة الأخرى التي تعد من الكلاسيكيات الروسية. وبعد فحص دقيق، تبيّن أن جميعها مزيفة بطريقة متقنة، إذ كانت تبدو أصلية لوجود أختام المكتبة وأرقام الجرد عليها.",
    "لم تكن مكتبة تارتو هي الضحية الوحيدة. فبعد أسابيع، اختفت أيضاً عشرة كتب نادرة من مكتبة جامعة تالين في العاصمة

In [ ]:
# For Translation

text = tokenizer.apply_chat_template(translation_messages, tokenize=False, add_generation_prompt=True)

model_inputs = tokenizer([text],  return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=1024,
                               do_sample = False,  top_k=None, temperature=None, top_p=None)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

translation_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(translation_text)

```json
{
  "translation_title": "Two Estonian Students Steal Eighteenth Century Russian Literature Books",
  "translation_content": "In April/May 2022, two students entered the library at Tallinn University in Estonia and requested to read eight books on Russian artists from the 18th century: the poet and playwright Alexander Pushkin, and the famous Russian writer with Ukrainian origins Nikolai Gogol."
}
```


## Evaluation Groq

In [ ]:
from google.colab import userdata
from groq import Groq

groq_client = Groq(api_key = userdata.get('groq'))
llama_model_id = "llama-3.1-8b-instant"

In [ ]:
chat_completion = groq_client.chat.completions.create(messages= details_extraction_messages, model = llama_model_id, temperature=0.2)
llama_details = chat_completion.choices[0].message.content
print(llama_details)

{
  "article_title": "سرقة كتب قديمة من مكتبات أوروبية",
  "article_date": "2023-10-01T00:00:00",
  "article_keywords": [
    "سرقة كتب",
    "مكتبات أوروبية",
    "كتب قديمة",
    "جورجيا",
    "إستونيا",
    "لاتفيا",
    "فرنسا",
    "سويسرا",
    "فنلندا"
  ],
  "article_summary": [
    "تم سرقة كتب قديمة من مكتبات أوروبية",
    "تم اكتشاف سرقة كتب من مكتبة جامعة تارتو في إستونيا",
    "تم اكتشاف سرقة كتب من مكتبة جامعة تالين في العاصمة الإستونية",
    "تم اكتشاف سرقة كتب من مكتبة جامعة وارسو في بولندا",
    "تم اكتشاف سرقة كتب من مكتبات أخرى في أوروبا",
    "تم اعتقال تسعة أشخاص بتهمة سرقة الكتب",
    "تم اكتشاف أن السارقين كانوا من جورجيا",
    "تم اكتشاف أن السارقين كانوا يعملون معًا",
    "تم اكتشاف أن السارقين كانوا يستخدمون تقنيات متقدمة لسرقة الكتب",
    "تم اكتشاف أن السارقين كانوا يبيعون الكتب المسرقة على الإنترنت"
  ],
  "article_category": "ثقافة",
  "article_entity": [
    {
      "entity_value": "بيكا تسيريكيدزه",
      "entity_type": "person-male"
    },
    {
      "

In [ ]:
chat_completion = groq_client.chat.completions.create(messages= translation_messages, model = llama_model_id, temperature=0.2)
llama_translation = chat_completion.choices[0].message.content
print(llama_translation)

{
  "translation_title": "Estonian University Libraries Hit by Rare Book Heists",
  "translation_content": "In April 2022, two men entered the University of Tartu library in Estonia, requesting to view eight books by 19th-century Russian artists, including poet and playwright Alexander Pushkin and famous Russian writer of Ukrainian origin, Nikolai Gogol. The men spoke in Russian, assuring the librarian that the younger of the two was conducting research to complete his studies in the United States. Three months later, library staff noticed that two of the eight books had been replaced with fake copies. The library's records showed that the original books had last been borrowed by the same two men in April. The library administrators quickly inspected the six other books, which were all classics of Russian literature. After a thorough examination, it was found that all of them were fake, with a high level of sophistication, as they looked original due to the presence of library stamps a

## Knowledge Distilation

In [ ]:
raw_data_path = join(data_dir, "datasets", "news-sample.jsonl")

raw_data = []

for line in open(raw_data_path):
  if line.strip() =="":
    continue
  raw_data.append(json.loads(line.strip()))

random.Random(44).shuffle(raw_data)
print(f"Raw data: {len(raw_data)} \n {raw_data[0]['content']}")

Raw data: 2400 
 قالت هيومن منظمة رايتس ووتش إن جماعة إم 23 المسلحة المدعومة من رواندا نفذت إعدامات بإجراءات موجزة وتجنيدا قسريا للمدنيين في شرق جمهورية الكونغو الديمقراطية. 
 ووفق المنظمة، فإن الجيش الكونغولي يرد على هجوم حركة إم 23 من خلال التعاون مع المليشيات العرقية ذات السجلات المسيئة. 
 وقال توماس فيسي، الباحث الأول في المنظمة، إن متمردي حركة إم 23 المدعومة من رواندا في شمال كيفو يتركون وراءهم سلسلة متزايدة من جرائم الحرب ضد المدنيين. 
 وطالب رواندا بإنهاء دعمها العسكري للحركة، والقوات الحكومية الكونغولية بإعطاء الأولوية لحماية المدنيين والتوقف عن استخدام المليشيات المسيئة كقوات بالوكالة. 
 وتوفر التحقيقات الأخيرة التي أجراها فريق خبراء الأمم المتحدة المعني بالكونغو، وكذلك أبحاث هيومن رايتس ووتش، صورا فوتوغرافية مهمة وغيرها من الأدلة على أن رواندا لا تقدم الدعم اللوجستي لحركة إم 23 فحسب، بل إن القوات الرواندية تدعم أو تقاتل إلى جانب الجماعة المسلحة داخل الكونغو الديمقراطية. ونفت الحكومة الرواندية دعم متمردي حركة إم 23. 
 أجبرت الأعمال العدائية المتجددة من قبل حركة إم 23 والجيش ال

In [ ]:
model_id = "llama-3.1-8b-instant"
price_per_1m_input_tokens = 0.150
price_per_1m_output_tokens = 0.600

prompt_tokens = 0
completion_tokens = 0
ix = 0

output_save = join(data_dir, "datasets", "sft.jsonl")


for article in tqdm(raw_data):
  details_extraction = [
      {
        "role":"system",
        "content":"/n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role":"user",
        "content":"\n".join([
            "## Article:",
            article["content"].strip(),
            "",

            "## Pydantic Scheme:",
            json.dumps(ArticleDetails.model_json_schema(), ensure_ascii=False),
            "",

            "## Article Details:",
            "```json"
        ])
    }
  ]
  try:
    extract_response = groq_client.chat.completions.create(
            messages=details_extraction,
            model=model_id,
            temperature=0.1
        )
  except Exception as e:
    print(e)
    break

  if extract_response.choices[0].finish_reason != "stop":
    prompt_tokens += extract_response.usage.prompt_tokens
    continue

  llm_response = extract_response.choices[0].message.content
  llm_response_dict = repaire_json(llm_response)

  if not llm_response_dict:
    continue

  with open(output_save, "a", encoding="utf-8") as sft:
    sft.write(json.dumps({
        "id":ix,
        "article":article["content"].strip(),
        "task": "Extrat the article details into a JSON.",
        "output_schema": json.dumps(ArticleDetails.model_json_schema(), ensure_ascii=False),
        "response":llm_response_dict
    }, ensure_ascii = False, default = str) + "\n")

  ix+=1
  prompt_tokens += extract_response.usage.prompt_tokens
  completion_tokens += extract_response.usage.completion_tokens

  if(x % 3) == 0 :
    cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
    cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
    total_cost = cost_input + cost_output
    print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")


In [ ]:
model_id = "llama-3.1-8b-instant"
price_per_1m_input_tokens = 0.150
price_per_1m_output_tokens = 0.600

prompt_tokens = 0
completion_tokens = 0
ix = 0

output_save = join(data_dir, "datasets", "sft.jsonl")

for article in tqdm(raw_data):
    for targeted_lang in ["English", "French"]:
        translation_role = [
            {
              "role": "system",
              "content": "\n".join([
                  "You are a professional translator.",
                  "You will be provided by an Arabic text.",
                  "You have to translate the text into the `Targeted Language`.",
                  "Follow the provided Scheme to generate a JSON",
                  "Do not generate any introduction or conclusion."
                ])
          },
          {
              "role":"user",
              "content":"\n".join([
                  "## Article:",
                  article["content"].strip(),
                  "",

                  "## Pydantic Scheme:",
                  json.dumps(Translated_Article.model_json_schema(), ensure_ascii=False),
                  "",

                  "## Targeted Language:",
                  targeted_lang,
                  "",

                  "## Article Details:",
                  "```json"
              ])
          }
        ]
        try:
          translate_response = groq_client.chat.completions.create(
                  messages=translation_role,
                  model=model_id,
                  temperature=0.1
              )
        except Exception as e:
          print(e)
          break

        if translate_response.choices[0].finish_reason != "stop":
          prompt_tokens += translate_response.usage.prompt_tokens
          continue

        llm_translate = translate_response.choices[0].message.content
        llm_translate_dict = repaire_json(llm_translate)

        if not llm_translate_dict:
          continue

        with open(output_save, "a", encoding="utf-8") as sft:
          sft.write(json.dumps({
              "id":ix,
              "article":article["content"].strip(),
              "task": f"You have to translate the story content into {targeted_lang} associated with a title into a JSON.",
              "output_schema": json.dumps(Translated_Article.model_json_schema(), ensure_ascii=False),
              "response":llm_translate_dict
          }, ensure_ascii = False, default = str) + "\n")

        ix+=1
        prompt_tokens += translate_response.usage.prompt_tokens
        completion_tokens += translate_response.usage.completion_tokens

        if(x % 3) == 0 :
          cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
          cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
          total_cost = cost_input + cost_output
          print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")


## Structure Output on Dataset


In [ ]:
sft_data_path = join(data_dir, "datasets", "sft.jsonl")
llm_ft_data = []


system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])


for line in open(sft_data_path):
  if line.strip() == "":
    continue
  rec = json.loads(line.strip())

  llm_ft_data.append({
      "system": system_message,
      "instruction": "\n".join({
            "# Article:",
            rec["story"],

            "# Task:",
            rec["task"],

            "# Output Scheme:",
            rec["output_scheme"],
            "",

            "# Output JSON:",
            "```json"
      }),
      "input": "",
      "output":"\n".join({
          "```json",
          json.dumps(rec["response"], ensure_ascii = False, default = str),
          "```"
      }),
      "history":[]
        })

random.Random(44).shuffle(llm_ft_data)

## Split & Train data

In [ ]:
len(llm_ft_data)

# Splitting data
train_sample_size = 2700
train_ds = llm_ft_data[:train_sample_size]
eval_ds = llm_ft_data[train_sample_size:]

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
  json.dump(train_ds, dest, ensure_ascii = False, default = str)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding = "utf8") as dest:
  json.dump(eval_ds, dest, ensure_ascii = False, default = str)

In [ ]:
join(data_dir)

'/gdrive/MyDrive/FineTuning_fromScratch'

## Fine-Tuning


In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/articl_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: articls_finetune_train
eval_dataset: articls_finetune_val
template: qwen
cutoff_len: 3500
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: /gdrive/MyDrive/FineTuning_fromScratch/models/
logging_steps: 10
save_steps: 500
plot_loss: true


### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: news-finetune-llamafactory


Writing /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/articl_finetune.yaml

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = None
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
finetuned_model_id = "/gdrive/MyDrive/FineTuning_fromScratch/models"
finetuned_model_id

'/gdrive/MyDrive/FineTuning_fromScratch/models'

In [ ]:
finetuned_model_id = "/gdrive/MyDrive/FineTuning_fromScratch/models"
model.load_adapter(finetuned_model_id)

In [ ]:
# For Details Extraction & Translation after Fine-Tuning

def generate_resp(messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

response = generate_resp(details_extraction_messages)

In [ ]:
repaire_json(response)

{'article_title': 'سرقة كتب قديمة من مكتبات جامعة تارتو ووالين في إستونيا',
 'article_date': '2023-10-01',
 'article_keywords': ['سرقة',
  'كتب قديمة',
  'مكتبات',
  'جامعة تارتو',
  'جامعة وارسو'],
 'article_summary': ['دخل شخصان مكتبة جامعة تارتو في إستونيا وطلبا الاطلاع على ثمانية كتب.',
  'استبدال كتابين من الكتب الثمانية بنسخ مزيفة بعد ثلاثة أشهر.',
  'اختفت أيضاً عشرة كتب نادرة من مكتبة جامعة تالين في العاصمة الإستونية.',
  'تم إطلاق وكالة الاتحاد الأوروبي للتعاون في مجال إنفاذ القانون تحقيقاً يسمى بـ"عملية بوشكين".',
  'تم اعتقال تسعة من المشتبه بهم، جميعهم يحملون الجنسية الجورجية.'],
 'article_category': 'art',
 'article_entity': [{'entity_value': 'جامعة تارتو',
   'entity_type': 'organization'},
  {'entity_value': 'ألكسندر بوشكين', 'entity_type': 'person-male'},
  {'entity_value': 'نيكولاي غوغول', 'entity_type': 'person-male'},
  {'entity_value': 'جامعة وارسو', 'entity_type': 'organization'},
  {'entity_value': 'ماتي', 'entity_type': 'person-male'},
  {'entity_value': 'آنا غوغ

## vLLM

In [7]:
# Run this on termial
#vllm serve "Qwen/Qwen2.5-1.5B-Instruct" --dtype=half --gpu-memory-utilization 0.8 --max_lora_rank 64 --enable-lora --lora-modules articls-lora="/gdrive/MyDrive/FineTuning_fromScratch/models"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

prompt = tokenizer.apply_chat_template(translation_messages, tokenize=False, add_generation_prompt=True)

In [9]:
vllm_model_id = "articls-lora"

llm_response = requests.post("http://localhost:8000/v1/completions", json={
    "model": vllm_model_id,
    "prompt": prompt,
    "max_tokens": 1000,
    "temperature": 0.3
})

llm_response.json()

{'id': 'cmpl-0a7aec40e6e14175a4e7ab15059f1320',
 'object': 'text_completion',
 'created': 1756914258,
 'model': 'articls-lora',
 'choices': [{'index': 0,
   'text': '```json{"translated_title": "The Case of Book Thieves: A Heinous Crime in Europe", "translated_content": "In April 2022, two individuals entered the University of Tartu library in Estonia and requested to view eight books by Russian 19th-century artists: the Russian playwright and theater writer Alexander Pushkin, and the famous Russian-born Ukrainian novelist Nikolai Gogol.\\n\\nThe two men spoke Russian, confirming to the library staff that the younger of the two was conducting research to complete his studies in the United States. Three months later, library staff noticed the replacement of two of the eight books with copies. The library records revealed that the original copies had last been in the possession of the same men in April.\\n\\nThe library\'s custodians quickly checked the six other books that are considere

## Load HTML Test LLM

In [14]:
%%writefile locust.py

import random
import json
from locust import HttpUser, task, between, constant
from transformers import AutoTokenizer
from faker import Faker

fake = Faker('ar')

class Load_Test(HttpUser):
    wait_time = between(1, 3)

    @task
    def post_test_llm(self):
        model_id = "articls-lora"
        prompt = fake.text(max_nb_chars=random.randint(150, 200))

        message = {
            "model": model_id,
            "prompt": prompt,
            "max_tokens": 512,
            "temperature": 0.3
        }

        llm_response = self.client.post("/v1/completions", json=message)



Writing locust.py


In [ ]:
!locust --headless -f locust.py --host=http://localhost:8000 -u 20 -r 1 -t "60s" --html=Test_Speed_LLM.html